In [30]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

In [31]:
#Loading data
df_anz = pd.read_excel('ANZ synthesised transaction dataset.xlsx', sheet_name= 'DSynth_Output_100c_3m_v3')
df_anz = df_anz.astype({"card_present_flag":'bool'})
df_anz.head(1)

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,True,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit


In [32]:
#Create 100 customets with their pay/salary table:
salary = df_anz[df_anz['txn_description'] == 'PAY/SALARY'].groupby(['customer_id','first_name','txn_description'])['amount'].agg(['count','sum']).sort_values(by='sum',ascending=0)
salary = salary.reset_index()
salary
#100 customer_id = 100 accounts = 100 first_name = 100 long_lat transactions in last 3 months

,customer_id,first_name,txn_description,count,sum
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10
5,CUS-2031327464,Michael,PAY/SALARY,13,29644.68
6,CUS-527400765,Natasha,PAY/SALARY,7,29512.28
7,CUS-3336454548,Charles,PAY/SALARY,7,28925.61
8,CUS-883482547,Michelle,PAY/SALARY,7,27842.22
9,CUS-1462656821,Isaiah,PAY/SALARY,7,27327.65


In [33]:
#Define payment type for each customer:
salary['payment_type'] = 'Quarterly'
salary['payment_type'] [salary['count'] == 3 ] = 'Monthly'
salary['payment_type'] [salary['count'] == 6 ] = 'Fortnightly'
salary['payment_type'] [salary['count'] == 7 ] = 'Fortnightly'
salary['payment_type'] [salary['count'] == 13 ] = 'Weekly'
salary['payment_type'] [salary['count'] == 14 ] = 'Weekly'
salary

C:\Users\My Computer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\My Computer\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\My Computer\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users

,customer_id,first_name,txn_description,count,sum,payment_type
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly
5,CUS-2031327464,Michael,PAY/SALARY,13,29644.68,Weekly
6,CUS-527400765,Natasha,PAY/SALARY,7,29512.28,Fortnightly
7,CUS-3336454548,Charles,PAY/SALARY,7,28925.61,Fortnightly
8,CUS-883482547,Michelle,PAY/SALARY,7,27842.22,Fortnightly
9,CUS-1462656821,Isaiah,PAY/SALARY,7,27327.65,Fortnightly


In [34]:
#Calculate Average Income for each customer:
salary['avg_income']= salary['sum']/salary['count']
salary.head(3)

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76


In [35]:
#Calculate Annual Salary after tax 
def get_status(salary):
    if salary['payment_type'] == 'Weekly':
        return salary['avg_income']*52
    elif salary['payment_type'] == 'Fortnightly':
        return salary['avg_income']*26
    elif salary['payment_type'] == 'Monthly':
        return salary['avg_income']*12
    else:
        return salary['avg_income']*4

salary['salary_after_tax_p.a'] = salary.apply(get_status, axis = 1)
salary['salary_after_tax_p.a'] = np.around(salary['salary_after_tax_p.a']).astype('int64')
salary.head()

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01,134577
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76,132328
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly,2443.24,127048
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly,4405.30,114538


In [36]:
# Calculate weekly average income to find out their weekly witholding tax
def get_status(salary):
    if salary['payment_type'] == 'Weekly':
        return salary['avg_income']
    elif salary['payment_type'] == 'Fortnightly':
        return salary['avg_income']/2
    elif salary['payment_type'] == 'Monthly':
        return salary['avg_income']*3/12
    else:
        return salary['avg_income']/13

salary['weekly_income'] = salary.apply(get_status, axis = 1)
salary.head()

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a,weekly_income
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344,679.690769
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01,134577,2588.010000
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76,132328,2544.760000
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly,2443.24,127048,2443.240000
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly,4405.30,114538,2202.650000


In [37]:
#Round numbers in numpy
salary['weekly_income'] = np.around(salary['weekly_income'])
salary.head(1)

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a,weekly_income
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344,680.0


In [38]:
salary['weekly_income']=salary['weekly_income']+0.99
salary.head()

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a,weekly_income
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344,680.99
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01,134577,2588.99
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76,132328,2545.99
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly,2443.24,127048,2443.99
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly,4405.30,114538,2203.99


In [39]:
# Calculate Weekly Tax Withholding Amount by applying 'Schedule 1' ATO formula
salary['weekly_tax_withholding'] = np.where(salary['weekly_income']<72, salary['weekly_income'] *0.19 - 0.19, 
        np.where(salary['weekly_income'] < 361, salary['weekly_income'] *0.2342 - 3.2130,
        np.where(salary['weekly_income'] < 932, salary['weekly_income']*0.3477- 44.2476,
        np.where(salary['weekly_income']< 1380, salary['weekly_income']*0.3450- 41.7311,
        np.where(salary['weekly_income'] < 3111, salary['weekly_income'] *0.3900- 103.8657,
                 salary['weekly_income'] *0.47 - 352.7888)))))
salary.head()

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a,weekly_income,weekly_tax_withholding
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344,680.99,192.532623
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01,134577,2588.99,905.840400
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76,132328,2545.99,889.070400
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly,2443.24,127048,2443.99,849.290400
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly,4405.30,114538,2203.99,755.690400


In [40]:
#Calculate Full Annual Salary:
salary['annual_salary'] = np.where(salary['payment_type']== 'Weekly', (salary['weekly_income']+salary['weekly_tax_withholding'])*52, 
        np.where(salary['payment_type']== 'Fortnightly', (salary['avg_income']+(salary['weekly_tax_withholding']*2))* 26,
        np.where(salary['payment_type']== 'Monthly', (salary['avg_income']+(salary['weekly_tax_withholding']*13/3))* 12,
                 (salary['avg_income']+(salary['weekly_tax_withholding']))*4)))
salary['annual_salary'] = np.around(salary['annual_salary']).astype('int64')
salary

,customer_id,first_name,txn_description,count,sum,payment_type,avg_income,salary_after_tax_p.a,weekly_income,weekly_tax_withholding,annual_salary
0,CUS-1816693151,Tim,PAY/SALARY,4,35343.92,Quarterly,8835.98,35344,680.99,192.532623,36114
1,CUS-3142625864,Ricky,PAY/SALARY,13,33644.13,Weekly,2588.01,134577,2588.99,905.840400,181731
2,CUS-2738291516,Kenneth,PAY/SALARY,13,33081.88,Weekly,2544.76,132328,2545.99,889.070400,178623
3,CUS-2155701614,Linda,PAY/SALARY,13,31762.12,Weekly,2443.24,127048,2443.99,849.290400,171251
4,CUS-261674136,Kenneth,PAY/SALARY,7,30837.10,Fortnightly,4405.30,114538,2203.99,755.690400,153834
5,CUS-2031327464,Michael,PAY/SALARY,13,29644.68,Weekly,2280.36,118579,2280.99,785.720400,159469
6,CUS-527400765,Natasha,PAY/SALARY,7,29512.28,Fortnightly,4216.04,109617,2108.99,718.640400,146986
7,CUS-3336454548,Charles,PAY/SALARY,7,28925.61,Fortnightly,4132.23,107438,2066.99,702.260400,143956
8,CUS-883482547,Michelle,PAY/SALARY,7,27842.22,Fortnightly,3977.46,103414,1989.99,672.230400,138370
9,CUS-1462656821,Isaiah,PAY/SALARY,7,27327.65,Fortnightly,3903.95,101503,1952.99,657.800400,135708


In [41]:
#Top 10 person with highest paid:
salary = salary.sort_values(by=['date'])
salary.head(10)

KeyError: 'date'